In [6]:
import xml.etree.ElementTree as ET
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import os
import subprocess
from shapely.geometry import Polygon, LineString, Point
import sumolib
import random

## Step 1: Create Network
1.	Download network from OSM
2.  At the intersection of Rheem Blvd and Moraga Rd, join the two junctions
3.	Make traffic lights actuated: netconvert -s moraga.net.xml --o moraga_actuated.net.xml --tls.rebuild --tls.default-type actuated
4.	Find and replace: traffic_light -> traffic_light_right_on_red
5.	Make another copy of the network
6. Tidy the network as needed
6.  Open the network in netedit. Find the Moraga Way edges and change them all the outbound edges to have 2 lanes instead of 1. Save this as moraga_extralane.net.xml

In [7]:
# !netconvert -s moraga_left.net.xml --o moraga_actuated.net.xml --tls.rebuild --tls.default-type actuated

## Step 2: Process census blocks and evacuation zones

In [8]:
# get Moraga census blocks
ca_census_blocks = gpd.read_file("tl_2020_06_tabblock20/tl_2020_06_tabblock20.shp")
ca_places = gpd.read_file("tl_2024_06_place/tl_2024_06_place.shp")
moraga_shape = ca_places[ca_places.NAME == "Moraga"]

moraga_cb = gpd.sjoin(ca_census_blocks, moraga_shape, how="inner", predicate="within")

moraga_cb = moraga_cb[["GEOID20", "HOUSING20", "POP20", "geometry"]]

moraga_cb.to_crs("epsg:4326", inplace=True)

In [9]:
# calculate distances to freeway entrance ramps
G = ox.graph_from_bbox(
    [-122.18944166245377, 37.80785236571763, -122.08530586034833, 37.901926337196215],
    network_type="drive",
)
orinda_fwy_coords = 37.87837782473702, -122.1834439745312
lafayette_fwy_coords = 37.89415711398768, -122.11658287702524
orinda_ramp = ox.nearest_nodes(G, orinda_fwy_coords[1], orinda_fwy_coords[0])
lafayette_ramp = ox.nearest_nodes(G, lafayette_fwy_coords[1], lafayette_fwy_coords[0])
moraga_cb["centroid_node"] = ox.nearest_nodes(
    G, moraga_cb.centroid.x, moraga_cb.centroid.y
)

moraga_cb["lafayette_dist"] = moraga_cb["centroid_node"].apply(
    lambda node: nx.shortest_path_length(
        G, source=node, target=lafayette_ramp, weight="length"
    )
)

moraga_cb["orinda_dist"] = moraga_cb["centroid_node"].apply(
    lambda node: nx.shortest_path_length(
        G, source=node, target=orinda_ramp, weight="length"
    )
)

/var/folders/rr/k04wzv7j0psdkxfsdqxwykn80000gn/T/ipykernel_62271/1832472875.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  G, moraga_cb.centroid.x, moraga_cb.centroid.y


In [10]:
# join evac data with census data
evac_zones = gpd.read_file("evacuation_zones_shp/evac_zones.shp")
evac_zones = evac_zones[
    ["zone_name", "zone_descr", "zone_speci", "geometry", "est_popula"]
]

# join evacuation zone data
zone_info = pd.read_excel("EvacuationZonePDFs/EvacuationZonesSpreadsheet.xlsx")
zone_info = zone_info[["Zone Name", "JoinID", "Via", "Via 2"]]

evac_zones = evac_zones.merge(
    zone_info, left_on="zone_name", right_on="JoinID", how="left"
)

evac_zones.to_crs(moraga_cb.crs, inplace=True)

In [11]:
# find centroids of evac zones
# calculate distance between centroid and fire start point
oaklandHillsStart = gpd.GeoSeries(
    Point(-122.22213416497657, 37.86114107653203), crs="4326"
).to_crs(evac_zones.crs)

merrillStart = gpd.GeoSeries(Point(-122.103867, 37.828267), crs="4326").to_crs(
    evac_zones.crs
)

buckinghamStart = gpd.GeoSeries(Point(-122.12351, 37.86339), crs="4326").to_crs(
    evac_zones.crs
)

evac_zones["distanceTo1991"] = evac_zones.centroid.distance(oaklandHillsStart.iloc[0])
evac_zones["distanceToMerrill"] = evac_zones.centroid.distance(merrillStart.iloc[0])
evac_zones["distanceToBuckingham"] = evac_zones.centroid.distance(
    buckinghamStart.iloc[0]
)

/var/folders/rr/k04wzv7j0psdkxfsdqxwykn80000gn/T/ipykernel_62271/2602838179.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  evac_zones["distanceTo1991"] = evac_zones.centroid.distance(oaklandHillsStart.iloc[0])
/var/folders/rr/k04wzv7j0psdkxfsdqxwykn80000gn/T/ipykernel_62271/2602838179.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  evac_zones["distanceTo1991"] = evac_zones.centroid.distance(oaklandHillsStart.iloc[0])
/var/folders/rr/k04wzv7j0psdkxfsdqxwykn80000gn/T/ipykernel_62271/2602838179.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  evac_zones["dist

In [12]:
# determine zone stage orders based on fire distance 

for i in range(2, 11):
    evac_zones[f"distance-rank-merrill-{i}"] = pd.qcut(
        evac_zones["distanceToMerrill"], q=i, labels=list(range(1, i + 1))
    ).astype(int)

for i in range(2, 11):
    evac_zones[f"distance-rank-buckingham-{i}"] = pd.qcut(
        evac_zones["distanceToBuckingham"], q=i, labels=list(range(1, i + 1))
    ).astype(int)

In [13]:
# Create centroids
moraga_cb_centroids = moraga_cb.copy()
moraga_cb_centroids["geometry"] = moraga_cb_centroids.centroid

# Spatial join
gdf_joined = gpd.sjoin(moraga_cb_centroids, evac_zones, how="left", predicate="within")

# Merge evacuation zone attributes back to the original blocks
evac_zone_columns = evac_zones.columns.difference(["geometry"])
moraga_cb = moraga_cb.merge(
    gdf_joined[evac_zone_columns.union(["GEOID20"])],
    on="GEOID20",
    how="left",
)

moraga_cb = gpd.GeoDataFrame(moraga_cb, geometry="geometry", crs=moraga_cb.crs)

/var/folders/rr/k04wzv7j0psdkxfsdqxwykn80000gn/T/ipykernel_62271/3353637241.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  moraga_cb_centroids["geometry"] = moraga_cb_centroids.centroid


In [14]:
# for zones with multiple exits, calculate which exit is closest
moraga_cb["closest_exit"] = moraga_cb.apply(
    lambda row: "Orinda" if row["orinda_dist"] < row["lafayette_dist"] else "Lafayette",
    axis=1,
)

moraga_cb["exit"] = np.where(
    moraga_cb["Via 2"].notnull() & (moraga_cb["closest_exit"] == "Lafayette"),
    "Moraga Rd",
    moraga_cb["Via"],
)

if not os.path.exists("MoragaCensusBlocks"):
    os.makedirs("MoragaCensusBlocks")

moraga_cb.to_file("MoragaCensusBlocks/moraga_cb.shp")

/var/folders/rr/k04wzv7j0psdkxfsdqxwykn80000gn/T/ipykernel_62271/1934198706.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  moraga_cb.to_file("MoragaCensusBlocks/moraga_cb.shp")
/opt/anaconda3/envs/urbsim/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'centroid_node' to 'centroid_n'
  ogr_write(
/opt/anaconda3/envs/urbsim/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'lafayette_dist' to 'lafayette_'
  ogr_write(
/opt/anaconda3/envs/urbsim/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'orinda_dist' to 'orinda_dis'
  ogr_write(
/opt/anaconda3/envs/urbsim/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'distance-rank-buckingham-10' to 'distance-r'
  ogr_write(
/opt/anaconda3/envs/urbsim/lib/python3.12/site-packages/pyogrio/raw.

**Start from here if running from GitHub clone**

## Step 3: Create TAZ file

In [15]:
# Convert Census Blocks shapefile to XML for use in SUMO
!polyconvert --net-file moraga_actuated_adjusted.net.xml \
  --shapefile-prefixes MoragaCensusBlocks/moraga_cb \
  --shapefile.id-column GEOID20 \
  --shapefile.add-param true \
  -o MoragaCensusBlocks.xml

Success.


In [16]:
# convert census blocks XML to TAZ file
!python $SUMO_HOME/tools/edgesInDistricts.py -n moraga_actuated_adjusted.net.xml -t MoragaCensusBlocks.xml -o districts.taz.xml -w 

## Step 4: Edit TAZs
- Create a TAZ for each exit edge
- Adjust the weights of each TAZ source/sink so that vehicles don't generate on private roads

In [17]:
# move exit edges to their own TAZ

target_edges = {
    "Moraga Rd": "-7880646#0",
    "Rheem Blvd": "-7878367#0",
    "Moraga Way": "-656033299",
    "St Mary's Rd": "7867780",
}

ns = {"xsi": "http://www.w3.org/2001/XMLSchema-instance"}

# Parse XML
tree = ET.parse("districts.taz.xml")
root = tree.getroot()

# Remove target edges from all existing TAZs
for taz in root.findall("taz"):
    edges_attr = taz.get("edges", "")
    edges = edges_attr.split()
    filtered_edges = [e for e in edges if e not in target_edges.values()]
    if len(edges) != len(filtered_edges):
        taz.set("edges", " ".join(filtered_edges))

# Add new TAZs with the removed edges
for road, edge in target_edges.items():
    new_taz = ET.Element("taz", {"id": road, "color": "255,0,0", "edges": f"{edge}"})
    root.append(new_taz)

In [18]:
# create a df with weights for each edge in the network
# change all edge weights to 0 if edge_speed = 0 (or null), 1 otherwise

moraga_net = pd.read_xml("moraga_actuated_adjusted.net.xml", xpath=".//edge/lane")
moraga_net = moraga_net[~moraga_net["id"].str.contains(":")]
moraga_net["edge"] = moraga_net["id"].str[:-2]
vType = "passenger"
weights = []

for i, row in moraga_net.iterrows():
    if row["disallow"] is None:
        weight = 1 if row["allow"].find(vType) != -1 else 0
    elif row["allow"] is None:
        weight = 0 if row["disallow"].find(vType) != -1 else 1
    else:
        print(f"This row doesn't have allow info: {row}")
    weights.append(weight)

moraga_net["weights"] = weights

In [19]:
# loop through districts file, for each source/sink, change the weight to the corresponding weight in moraga_net df

for elem in root.findall(".//tazSource") + root.findall(".//tazSink"):
    id = elem.get("id")
    edges = moraga_net[moraga_net.edge == id]
    weight = "1" if sum(edges.weights) > 0 else "0"
    elem.set("weight", weight)

tree.write("districts_modified.taz.xml", encoding="UTF-8", xml_declaration=True)

## Step 5: Generate OD matrix
- produces OD matrix in Amitran format assuming 1 car per household

In [20]:
# Create OD Matrix in Amitran XML format
# Create the root XML element
root = ET.Element(
    "demand",
    {
        "xmlns:xsi": "https://www.w3.org/2001/XMLSchema-instance",
        "xsi:noNamespaceSchemaLocation": "https://sumo.dlr.de/xsd/amitran/od.xsd",
    },
)

actorConfig = ET.SubElement(root, "actorConfig", {"id": "1"})

timeSlice = ET.SubElement(
    actorConfig,
    "timeSlice",
    {
        "duration": "600000",
        "startTime": "0",  # this is a placeholder. timings will be edited later.
    },
)

from_column = "GEOID20"
count_column = "HOUSING20"

# Add TAZ relations, skipping blocks with 0 or missing households
for _, row in moraga_cb.iterrows():
    count = row[count_column]
    evac_zone = row.zone_name
    myexit = row.exit

    if pd.isna(count) or count == 0:
        continue  # Skip blocks with 0 or NaN households

    ET.SubElement(
        timeSlice,
        "odPair",
        {
            "origin": str(row[from_column]),
            "destination": f"{myexit}",
            "amount": str(int(count)),
        },
    )

tree = ET.ElementTree(root)
tree.write("amitran.xml", encoding="utf-8", xml_declaration=True)

## Step 6: Convert OD matrix to trips.xml


In [21]:
!od2trips -c od2tripsconfig.od2tcfg

Success.time 599.63


Create new housing scenario trips files

In [22]:
# Add 66 trips to edge 169253400#1 (School St edge)
schoolst_cb = "060133522021022"
# Parse XML
tree = ET.parse("trips.trips.xml")
root = tree.getroot()
ns = {"xsi": "http://www.w3.org/2001/XMLSchema-instance"}

# Add trip elements for the School St project
for i in range(0, 66):
    new_trip = ET.Element(
        "trip",
        {
            "id": f"new_{i}",
            "depart": "600",
            "from": "169253400#1",
            "to": target_edges["Moraga Way"],
            "fromTaz": schoolst_cb,
            "toTaz": "Moraga Way",
            "departLane": "free",
            "departSpeed": "max",
        },
    )
    root.append(new_trip)

tree.write("trips_newhousing.trips.xml", encoding="UTF-8", xml_declaration=True)

In [23]:
# create extreme housing scenario
general_plan_housing = gpd.read_file("new_housing/new_housing.shp")

# required_units = 1118 - 66 = 1052
# so if 19 sites, 55 trips at 12 sites and 56 trips at 7 sites
site_trips = list(np.repeat(55, 12)) + list(np.repeat(56, 7))
random.shuffle(site_trips)
general_plan_housing["trips"] = site_trips

closest_edges = []
net = sumolib.net.readNet("moraga_actuated_adjusted.net.xml")  # load once

for i, row in general_plan_housing.iterrows():
    radius = 1000
    x, y = net.convertLonLat2XY(row.geometry.x, row.geometry.y)
    edges = net.getNeighboringEdges(x, y, radius)

    # Filter: only edges that allow passenger traffic
    passenger_edges = [(edge, dist) for edge, dist in edges if edge.allows("passenger")]

    if passenger_edges:
        # Sort by distance
        dist, closestEdge = sorted(
            [(dist, edge) for edge, dist in passenger_edges], key=lambda x: x[0]
        )[0]
        closest_edges.append(closestEdge.getID())
    else:
        print(f"No passenger edge found for site {i}")
        closest_edges.append(None)

general_plan_housing["closest_edge"] = closest_edges

general_plan_housing = gpd.sjoin(
    general_plan_housing,
    moraga_cb[["GEOID20", "geometry", "exit"]],
    how="left",
    predicate="within",
)

In [24]:
tree = ET.parse("trips_newhousing.trips.xml")
root = tree.getroot()
ns = {"xsi": "http://www.w3.org/2001/XMLSchema-instance"}

# Add trip elements
for i, row in general_plan_housing.iterrows():
    for i in range(0, row["trips"]):
        new_trip = ET.Element(
            "trip",
            {
                "id": f"new_{row.id}_{i}",
                "depart": "600",
                "from": f"{row.closest_edge}",
                "to": target_edges[f"{row.exit}"],
                "fromTaz": f"{row.GEOID20}",
                "toTaz": f"{row.exit}",
                "departLane": "free",
                "departSpeed": "max",
            },
        )
        root.append(new_trip)

tree.write("trips_extremehousing.trips.xml", encoding="UTF-8", xml_declaration=True)

## Step 7: Convert trips.xml to routes with duarouter

1. create 4 different duarouter configuration files for each scenario: base lanes/no new housing, base lanes/new housing, extra lane on Moraga Way/no new housing, extra lane on Moraga Way/new housing

In [25]:
!duarouter -c duarcfg_baselanes_newhousing.duarcfg.xml

Success.up to time step: 600.12  (600.12/1800.00 = 33.34% done)       


In [26]:
!duarouter -c duarcfg_baselanes_nohousing.duarcfg.xml

Success.up to time step: 600.12  (600.12/1800.00 = 33.34% done)       


In [27]:
!duarouter -c duarcfg_baselanes_extremehousing.duarcfg.xml

Success.up to time step: 600.12  (600.12/1800.00 = 33.34% done)       


In [28]:
net = sumolib.net.readNet("moraga_actuated_extralane_adjusted.net.xml")
radius = 800
fireTrailLat = 37.87085387876746
fireTrailLon = -122.14182622566365
x, y = net.convertLonLat2XY(fireTrailLon, fireTrailLat)
edges = net.getNeighboringEdges(x, y, radius)
edge_ids = [edge.getID() for edge, dist in edges]

In [29]:
# reroute Campolindo neighborhood along Rim Trail
def adjustTrips(trips):
    # Parse the trips XML file
    tree = ET.parse(f"{trips}.trips.xml")
    root = tree.getroot()

    # Iterate through <trip> elements and modify `toTaz` where needed
    for trip in root.findall("trip"):
        from_edge = trip.get("from")
        cb = trip.get("fromTaz")
        zone = moraga_cb[moraga_cb["GEOID20"] == cb]["zone_name"].iloc[0]
        if (from_edge in edge_ids) and (zone == "MOR-E001"):
            trip.set("toTaz", "Rheem Blvd")
            trip.set("to", "-7878367#0")

    # Save modified XML back to file (overwrite or new file)
    tree.write(f"{trips}_extralane.trips.xml", encoding="UTF-8", xml_declaration=True)

adjustTrips("trips")
adjustTrips("trips_newhousing")

In [30]:
!duarouter -c duarcfg_extralane_newhousing.duarcfg.xml

Success.up to time step: 600.12  (600.12/1800.00 = 33.34% done)       


In [31]:
!duarouter -c duarcfg_extralane_nohousing.duarcfg.xml

Success.up to time step: 600.12  (600.12/1800.00 = 33.34% done)       


clear/create directories

In [32]:
# create directories for outputs
if not os.path.exists("unsorted_routes"):
    os.makedirs("unsorted_routes")

if not os.path.exists("sorted_route_files"):
    os.makedirs("sorted_route_files")

if not os.path.exists("sumo_configs"):
    os.makedirs("sumo_configs")

if not os.path.exists("summary_outputs"):
    os.makedirs("summary_outputs")

if not os.path.exists("summary_csvs"):
    os.makedirs("summary_csvs")

if not os.path.exists("sumo_logs"):
    os.makedirs("sumo_logs")

if not os.path.exists("edgedata_outputs"):
    os.makedirs("edgedata_outputs")

if not os.path.exists("tripinfo_outputs"):
    os.makedirs("tripinfo_outputs")

In [33]:
# clear folders
for file in os.listdir("summary_outputs"):
    os.remove(f"summary_outputs/{file}")
    print(f"removing {file}")

for file in os.listdir("sumo_configs"):
    os.remove(f"sumo_configs/{file}")
    print(f"removing {file}")

for file in os.listdir("summary_csvs"):
    os.remove(f"summary_csvs/{file}")
    print(f"removing {file}")

for file in os.listdir("sumo_logs"):
    os.remove(f"sumo_logs/{file}")
    print(f"removing {file}")

for file in os.listdir("edgedata_outputs"):
    os.remove(f"edgedata_outputs/{file}")
    print(f"removing {file}")

for file in os.listdir("tripinfo_outputs"):
    os.remove(f"tripinfo_outputs/{file}")
    print(f"removing {file}")

for file in os.listdir("unsorted_routes"):
    os.remove(f"unsorted_routes/{file}")
    print(f"removing {file}")

removing distance-rank-buckingham-6_0_150_baselanes_newhousing.rou.xml
removing distance-rank-buckingham-6_0_150_baselanes_extremehousing.rou.xml
removing distance-rank-buckingham-6_0_150_extralane_newhousing.rou.xml
removing distance-rank-buckingham-6_0_600_extralane_newhousing.rou.xml
removing distance-rank-buckingham-6_0_600_extralane_nohousing.rou.xml
removing distance-rank-buckingham-6_0_600_baselanes_nohousing.rou.xml
removing distance-rank-buckingham-6_0_1800_baselanes_newhousing.rou.xml
removing distance-rank-buckingham-6_0_1800_extralane_nohousing.rou.xml
removing distance-rank-buckingham-6_600_150_baselanes_newhousing.rou.xml
removing distance-rank-buckingham-6_0_1800_baselanes_nohousing.rou.xml
removing distance-rank-buckingham-6_0_600_baselanes_extremehousing.rou.xml
removing distance-rank-buckingham-6_0_1800_extralane_newhousing.rou.xml
removing distance-rank-buckingham-6_0_600_baselanes_newhousing.rou.xml
removing distance-rank-buckingham-6_600_150_baselanes_nohousing.rou

## Step 8: Adjust departure and notification timing
- Create route files for each simulation

In [34]:
# adjust the route timing based on staging and milling time spread
def adjustRouteTiming(sliceDuration, theta, k, routes, zone_order):
    tree = ET.parse(routes)
    root = tree.getroot()
    veh_count = len(root.findall("vehicle"))
    departure_times = np.random.gamma(shape=k, scale=theta, size=veh_count)

    total_times = []
    for elem in root.findall("./vehicle"):
        id = elem.get("id")
        geoid = elem.get("fromTaz")
        evac_zone = moraga_cb[moraga_cb.GEOID20 == geoid].zone_name.iloc[0]
        # set delay to last value of departure_times and remove from the list
        delay, departure_times = departure_times[-1], departure_times[:-1]
        notice_time = (
            evac_zones[evac_zones.zone_name == evac_zone][zone_order] - 1
        ) * sliceDuration  # time notice sent = stage number * staging time + milling time
        depart = notice_time.iloc[0] + delay
        total_times.append(depart)
        elem.set("depart", str(depart))

    tree.write(
        f"unsorted_routes/{zone_order}_{sliceDuration}_{theta}_{routes}",
        encoding="UTF-8",
        xml_declaration=True,
    )

    return total_times, departure_times

In [35]:
sliceDurationScenarios = [0, 600]  # simultaneous, 10 minutes
# k * theta =~ mean
k = 2  # shape, larger shape = more normal looking, smaller = more skewed right
thetas = [150, 600, 1800]  # means of 5 minutes, 20 minutes, and 1 hour

route_file_names = [
    "baselanes_nohousing.rou.xml",
    "baselanes_newhousing.rou.xml",
    "extralane_newhousing.rou.xml",
    "extralane_nohousing.rou.xml",
    "baselanes_extremehousing.rou.xml",
]

# 6 stages
zone_orders = ["distance-rank-buckingham-6", "distance-rank-merrill-6"]

# clear output folder
for file in os.listdir("sorted_route_files"):
    os.remove(f"sorted_route_files/{file}")
    print(f"removing {file}")

for seq in zone_orders:
    for sliceDuration in sliceDurationScenarios:
        for theta in thetas:
            for route_file in route_file_names:
                for i in range(0, 15):
                    adjustRouteTiming(sliceDuration, theta, k, route_file, seq)

                    sumo_home = os.environ["SUMO_HOME"]
                    input_file = (
                        f"unsorted_routes/{seq}_{sliceDuration}_{theta}_{route_file}"
                    )
                    output_file = f"sorted_route_files/sorted_{i}_{seq}_{sliceDuration}_{theta}_{route_file}"

                    cmd = [
                        "python",
                        os.path.join(sumo_home, "tools/route/sort_routes.py"),
                        input_file,
                        "-o",
                        output_file,
                    ]

                    subprocess.run(cmd, check=True)

removing sorted_0_distance-rank-buckingham-6_0_1800_extralane_newhousing.rou.xml
removing sorted_5_distance-rank-buckingham-6_0_1800_extralane_newhousing.rou.xml
removing sorted_8_distance-rank-buckingham-6_0_600_baselanes_newhousing.rou.xml
removing sorted_6_distance-rank-buckingham-6_0_150_extralane_newhousing.rou.xml
removing sorted_7_distance-rank-buckingham-6_0_600_extralane_nohousing.rou.xml
removing sorted_11_distance-rank-buckingham-6_0_150_extralane_nohousing.rou.xml
removing sorted_5_distance-rank-buckingham-6_0_1800_baselanes_extremehousing.rou.xml
removing sorted_10_distance-rank-buckingham-6_0_1800_extralane_newhousing.rou.xml
removing sorted_0_distance-rank-buckingham-6_0_150_baselanes_newhousing.rou.xml
removing sorted_10_distance-rank-buckingham-6_0_600_extralane_newhousing.rou.xml
removing sorted_10_distance-rank-buckingham-6_0_150_baselanes_extremehousing.rou.xml
removing sorted_10_distance-rank-buckingham-6_0_1800_baselanes_extremehousing.rou.xml
removing sorted_3_di

KeyboardInterrupt: 

## Step 9: Run sumo
- create sumocfg for each simulation and run the simulation

In [ ]:
for route_file in os.listdir("sorted_route_files"):
    # format: 0_distance-rank-buckingham-6_3600_150_extralane_newhousing
    tree = ET.parse("sumo1.sumocfg")
    root = tree.getroot()

    name = f"{route_file[7:-8]}"

    for elem in root.findall(".//additional-files"):
        elem.set("value", "../districts_modified.taz.xml")

    # change the configuration name
    for elem in root.findall(".//configuration-file"):
        elem.set("value", f"sumo_configs/{name}.sumocfg")

    # set route file
    for elem in root.findall(".//route-files"):
        elem.set("value", f"../sorted_route_files/{route_file}")

    # change output name to summary_routesfilename
    for elem in root.findall(".//summary-output"):
        elem.set("value", f"../summary_outputs/{name}_summary.xml")

    # if it's an extralane scenario change network to moraga_actuated_extralane.net.xml
    if "extralane" in route_file:
        for elem in root.findall(".//net-file"):
            elem.set("value", "../moraga_actuated_extralane_adjusted.net.xml")
    else:
        for elem in root.findall(".//net-file"):
            elem.set("value", "../moraga_actuated_adjusted.net.xml")

    for elem in root.findall(".//error-log"):
        elem.set("value", f"../sumo_logs/{name}")

    for elem in root.findall(".//edgedata-output"):
        elem.set("value", f"../edgedata_outputs/{name}")

    for elem in root.findall(".//tripinfo-output"):
        elem.set("value", f"../tripinfo_outputs/{name}")

    # Write the updated XML to a new file
    tree.write(f"sumo_configs/{name}.sumocfg", encoding="UTF-8", xml_declaration=True)

In [ ]:
for config in os.listdir("sumo_configs"):

    # if config[:-8] in os.listdir("tripinfo_outputs"):
    #    continue

    # if config == ".DS_Store":
    #  continue

    cmd = ["sumo", "-c", f"sumo_configs/{config}"]

    subprocess.run(
        cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
    )